This notebook is for visualizing point clouds with landmarks

In [2]:
import numpy as np
import pptk # Requires python 3.6!
import potpourri3d as pp3d
import csv
import glob
import os
from utils import eucl_dist
from pathlib import Path

In [10]:
# Visualize single prediction

fp = '/Users/carotenuto/Documents/GitHub/3d-facial-landmarks-omfs/diffusion-net/experiments/headspace_ldmks/no_op/preds/vis/pred_pt_17_2_333.txt_.txt'
pcl= np.loadtxt(fp,delimiter=',')
col = pcl[:,3:]
v = pptk.viewer(pcl[:,:3], col, show_axis=False)
v.set(point_size=1.0, show_axis=False, show_info=True)

In [8]:
# overlay prediction with ground truth from targets (hmap_per_class)

file_data_path='/Users/carotenuto/Master Radboud/MscProj/preds_pcl_all_c256_l10/preds/vis/pred_pt_01513.txt'
pcl= np.loadtxt(file_data_path,delimiter=',')
col1 = pcl[:,3:]
file_data_path2='/Users/carotenuto/Master Radboud/MscProj/preds_pcl_all_c256_l10/gt_vis/gt_pt_01513.txt'
pcl2= np.loadtxt(file_data_path2,delimiter=',')
col2 = pcl2[:,3:]

In [9]:
col = np.maximum(col1, col2)

In [10]:
#for i in range(len(col)):
#    if col[i,0] == 1 and col[i,1] == 1:
#        col[i,0] = 0.0

In [11]:
v = pptk.viewer(pcl[:,:3], col, show_axis=False)
v.set(point_size=0.5, show_axis=False, show_info=True)

In [21]:
# Visualize prediction against actual ground truth from ldmks.pkl
ldmks = np.load('/Users/carotenuto/Master Radboud/MscProj/manual_results/subjects_196_labelled/ldmks.pkl',
                allow_pickle=True)  # shape (samples, landmarks + 1, 3)
folder_num = '160'
for i in range(ldmks.shape[0]):
    if int(ldmks[i, 0, 0]) == int(folder_num):
        ldmks_idx = i
        break
ldmks_per_file = ldmks[ldmks_idx, 1:, :]  # shape (landmarks, 3)

file_data_path='/Users/carotenuto/Master Radboud/MscProj/manual_results/pcl_196_30k/preds/vis/pred_pt_00160_.txt'
pcl= np.loadtxt(file_data_path,delimiter=',')
col1 = pcl[:,3:]
pcl = pcl[:,:3]

LANDMARK_INDICES = [8, 27, 30, 31, 33, 35, 36, 39, 42, 45, 60, 64]
for i in range(len(ldmks_per_file)):
    if i in LANDMARK_INDICES:
        pcl = np.append(pcl, [ldmks_per_file[i]], axis=0)

        col1 = np.append(col1, [[0,1,0]], axis=0)

v = pptk.viewer(pcl, col1, show_axis=False)
v.set(point_size=0.5, show_axis=False, show_info=True)

In [15]:
file_data_path='/Users/carotenuto/Master Radboud/MscProj/preds_pcl_all_c256_l10/preds/vis/pred_01513.txt'
pcl= np.loadtxt(file_data_path,delimiter=',')
col1 = pcl[:,3:]
file_data_path2='/Users/carotenuto/Master Radboud/MscProj/preds_pcl_all_c256_l10/gt_vis/gt_01513.txt'
pcl2= np.loadtxt(file_data_path2,delimiter=',')
col2 = pcl2[:,3:]

In [7]:
col = np.zeros((col1.shape[0], col1.shape[1]))
for i in range(len(col)):
    col[i,2] = col1[i,1]

v = pptk.viewer(pcl[:,:3], col1, show_axis=False)
v.set(point_size=0.5, show_axis=False, show_info=True)

In [8]:
# Visualize Headspace labels against own manual labels

folder_number = '00091'
rootdir = '/Users/carotenuto/Master Radboud/MscProj/manual_labels/headspace0-101_ownlabels'
for filepath in glob.iglob(os.path.join(rootdir, folder_number, '*.obj')):
    verts, _ = pp3d.read_mesh(filepath)


rows = []
with open(os.path.join(rootdir, folder_number, 'ldmks.csv'), 'r') as f:
    csvreader = csv.reader(f)
    for row in csvreader:
        rows.append(row)

ldmks = rows[7:19]
ldmks_own = [row[1:4] for row in ldmks]

# only keep selected landmarks
LANDMARK_INDICES = [8, 27, 30, 31, 33, 35, 36, 39, 42, 45, 60, 64]
for filepath in glob.iglob(os.path.join(rootdir, folder_number, 'ldmks13*.txt')):
    ldmks_hs = np.loadtxt(filepath)
ldmks_hs = ldmks_hs.astype(int)
verts_vis = np.concatenate((verts, ldmks_own))

#c_verts = np.full(len(verts), False, dtype=bool)
c_mask = np.full(len(verts_vis), False, dtype=bool)
ldmks_hs = ldmks_hs[LANDMARK_INDICES]
c_mask[ldmks_hs-1] = True

print(np.unique(c_mask, return_counts=True))

#c_mask = verts_vis[ldmks_hs == True]
c = np.zeros((len(verts_vis), 3))
c[c_mask] = [1, 1, 0] # yellow, headspace labels


#c_ldmks_own = np.full(len(ldmks), True, dtype=bool)
#c_ldmks_hs = np.full(len(ldmks_per_file), True, dtype=bool)
#c_mask = np.concatenate((c_verts, c_ldmks, c_ldmks_hs))

 # green, own labels

c[len(c)-len(ldmks_own):] = [1, 0, 1] # pink, own labels

v = pptk.viewer(verts_vis, c, show_axis=False)
v.set(point_size=0.3, show_axis=False, show_info=True, bg_color=[1,1,1,0])

FileNotFoundError: [Errno 2] No such file or directory: '/Users/carotenuto/Master Radboud/MscProj/manual_labels/headspace0-101_ownlabels/00091/ldmks.csv'

In [4]:
file_data_path='/Users/carotenuto/Master Radboud/MscProj/subjects_1-10_copy/pcl/00001/130925161618.txt'
pcl= np.loadtxt(file_data_path,delimiter=',')
verts = pcl[:,:3]
col = pcl[:,3:]
v = pptk.viewer(verts, col, show_axis=False)
v.set(point_size=1.4, show_axis=False, show_info=True, bg_color=[1,1,1,0])